# Import Library

import tkinter as tk
from tkinter import messagebox, simpledialog
import mysql.connector

# Database Integration

In [2]:
# Establish a connection to the MySQL database
def create_connection():
    connection = mysql.connector.connect(
        host='localhost',
        user='root',  # Replace with your MySQL username
        password='17January2005',  # Replace with your MySQL password
        database='todolist'
    )
    return connection

# Task Management

In [3]:
# Function to add a task
def add_task():
    description = task_entry.get()
    if description:
        connection = create_connection()
        cursor = connection.cursor()
        cursor.execute("INSERT INTO tasks (description, status) VALUES (%s, %s)", (description, 'Incomplete'))
        connection.commit()
        cursor.close()
        connection.close()
        task_entry.delete(0, tk.END)
        load_tasks()
        messagebox.showinfo("Success", f'Task "{description}" added successfully!')
    else:
        messagebox.showwarning("Input Error", "Please enter a task description.")

# Function to load tasks from the database
def load_tasks():
    # Clear the current tasks in the frame
    for widget in task_frame.winfo_children():
        widget.destroy()
    
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM tasks")
    tasks = cursor.fetchall()
    
    for task in tasks:
        task_id = task[0]
        description = task[1]
        status = task[2]
        
        # Create a frame for each task
        task_item_frame = tk.Frame(task_frame, bg="#1b4d4f")
        task_item_frame.pack(fill="x", pady=5, padx=10)
        
        # Label to display task description
        task_label = tk.Label(task_item_frame, text=description, fg="white", bg="#1b4d4f", font=("Arial", 12))
        task_label.pack(side="left", padx=10, pady=5)

        # Button to mark task as complete
        complete_button = tk.Button(
            task_item_frame, text="Complete", bg="#1b4d4f", fg="white", font=("Arial", 10),
            command=lambda tid=task_id: mark_complete(tid)
        )
        complete_button.pack(side="right", padx=5)

        # Button to edit task
        edit_button = tk.Button(
            task_item_frame, text="Edit", bg="#1b4d4f", fg="white", font=("Arial", 10),
            command=lambda tid=task_id: edit_task(tid)
        )
        edit_button.pack(side="right", padx=5)
        
        # Button to delete task
        delete_button = tk.Button(
            task_item_frame, text="Delete", bg="#1b4d4f", fg="white", font=("Arial", 10),
            command=lambda tid=task_id: delete_task(tid)
        )
        delete_button.pack(side="right", padx=5)

    cursor.close()
    connection.close()

# Function to mark a task as complete
def mark_complete(task_id):
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute("UPDATE tasks SET status = %s WHERE id = %s", ('Complete', task_id))
    connection.commit()
    cursor.close()
    connection.close()
    load_tasks()
    messagebox.showinfo("Success", f'Task with ID {task_id} marked as complete!')

# Function to delete a task by ID
def delete_task(task_id):
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute("DELETE FROM tasks WHERE id = %s", (task_id,))
    connection.commit()
    cursor.close()
    connection.close()
    load_tasks()
    messagebox.showinfo("Success", f'Task with ID {task_id} deleted successfully!')

# Function to edit a task by ID
def edit_task(task_id):
    new_description = simpledialog.askstring("Edit Task", "Enter new description:")
    if new_description:
        connection = create_connection()
        cursor = connection.cursor()
        cursor.execute("UPDATE tasks SET description = %s WHERE id = %s", (new_description, task_id))
        connection.commit()
        cursor.close()
        connection.close()
        load_tasks()
        messagebox.showinfo("Success", f'Task with ID {task_id} updated successfully!')


In [ ]:
# User Interface (UI)

In [4]:
# Create the main window
root = tk.Tk()
root.title("To-Do App")
root.geometry("300x400")
root.config(bg="#2e2e2e")

# Title label
title_label = tk.Label(root, text="ToDo App", font=("Arial", 18, "bold"), bg="#2e2e2e", fg="white")
title_label.pack(pady=10)

# Create task input field
task_entry = tk.Entry(root, width=30, font=("Arial", 12))
task_entry.pack(pady=10)

# Add Task button
add_button = tk.Button(root, text="Add", command=add_task, font=("Arial", 10), bg="#1b4d4f", fg="white", width=10)
add_button.pack(pady=5)

# Frame to hold tasks
task_frame = tk.Frame(root, bg="#2e2e2e")
task_frame.pack(fill="both", expand=True, pady=10)

# Load existing tasks
load_tasks()

# Run the application
root.mainloop()
